In [ ]:
from nbdev import *
%nbdev_default_export weights
%load_ext autoreload
%autoreload 2

from utilities.ipynb_docgen import *
from nbdev.showdoc import show_doc

Cells will be exported to wtlike.weights,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Weights
> Load weighted data, combine with photon data

In [ ]:
%nbdev_export
import os, sys,  pickle, healpy
import numpy as np
from wtlike.config import *
#from wtlike.photon_data import *



In [ ]:
%nbdev_export
def check_weights(config, source):
    """
    Check that weights for the source are available: if so, return the weight file name
    
    - source -- A PointSource object with information on source location
    
    Returns the filepath to the file if successful, otherwise, print a message abount available files
    """
    weight_files = config.wtlike_data/'weight_files' 
    assert weight_files.is_dir(), f'Expect {weight_files} to be a directory'
    weight_file = weight_files/ (source.filename+'_weights.pkl')
    if not weight_file.exists():
        available = np.array(list(map(lambda p: p.name[:p.name.find('_weights')], 
                          weight_files.glob('*_weights.pkl'))))
        print(f'{source} not found in list of weight files at\n\t {weight_files}.\n Available:\n{available}',
             file = sys.stderr)
        return None
    return weight_file

In [ ]:

show_doc(check_weights)
config = Config(wtlike_data='~/wtlike_data')
if config.valid:
    print('Check not found')
    test_source = PointSource('test', (0,0))
    check_weights(config, test_source)
    good_source = PointSource('Geminga')
    print(f'{good_source} Should be found: file at {check_weights(config, good_source)} ')
    

<h4 id="check_weights" class="doc_header"><code>check_weights</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>check_weights</code>(**`config`**, **`source`**)

Check that weights for the source are available: if so, return the weight file name

- source -- A PointSource object with information on source location

Returns the filepath to the file if successful, otherwise, print a message abount available files

Check not found


Source "test" at: (l,b)=(0.000,0.000) not found in list of weight files at
	 /home/burnett/wtlike_data/weight_files.
 Available:
['011F-0138' '011F-0217' '011F-0263' '011H-0114' '011H-0116' '011S-0100'
 '03bN-0330' '03bS-0110' '04aF-0026' '04aS-0035' '204F-0050' '3C454.3'
 '3C_279' '504F-0472' '504F-0615' '504F-0753' '504F-0933' '504F-0961'
 '504F-1164' '504F-1322' '504F-1916' '504H-0058' '504H-0261' '504H-0309'
 '504H-0316' '504H-0369' '504H-0399' '504H-0499' '504N-0010' '504N-0717'
 '504N-0748' '504N-1076' '504N-1324' '504N-1409' '504N-1637' '504N-1663'
 '504N-1714' '504P-0428' '504P-0573' '504P-0614' '504P-0675' '504P-0777'
 '504P-0910' '504P-1095' '504P-1407' '504S-0546' '504S-0683' '504S-1612'
 '605F-0304' '605F-0537' '605F-0645' '605F-0679' '605F-0762' '605F-1060'
 '605F-1081' '605H-0133' '605H-0160' '605N-0485' '605N-0940' '605P-0285'
 '605P-0459' '605S-0359' '605S-0378' '605S-0391' '605S-0513' '998F-0474'
 '998N-0408' '998N-0517' 'B2_1520p31' 'BL_Lac' 'Geminga' 'P8010-0510'
 'P

Source "Geminga" at: (l,b)=(195.134,4.266) Should be found: file at /home/burnett/wtlike_data/weight_files/Geminga_weights.pkl 


In [ ]:
%nbdev_export
def _load_weights(config, filename, ):
    """Load the weight informaton

    filename: pickled dict with map info

    """
    # load a pickle containing weights, generated by pointlike
    assert os.path.exists(filename),f'File {filename} not found.'
    with open(filename, 'rb') as file:
        wtd = pickle.load(file, encoding='latin1')
    assert type(wtd)==dict, 'Expect a dictionary'
    test_elements = 'energy_bins pixels weights nside model_name radius order roi_name'.split()
    assert np.all([x in wtd.keys() for x in test_elements]),f'Dict missing one of the keys {test_elements}'
    if config.verbose>0:
        print(f'Load weights from file {os.path.realpath(filename)}')
        pos = wtd['source_lb']
        print(f'\tFound: {wtd["source_name"]} at ({pos[0]:.2f}, {pos[1]:.2f})')
    # extract pixel ids and nside used
    wt_pix   = wtd['pixels']
    nside_wt = wtd['nside']

    # merge the weights into a table, with default nans
    # indexing is band id rows by weight pixel columns
    # append one empty column for photons not in a weight pixel
    # calculated weights are in a dict with band id keys
    wts = np.full((32, len(wt_pix)+1), np.nan, dtype=np.float32)
    weight_dict = wtd['weights']
    for k in weight_dict.keys():
        t = weight_dict[k]
        if len(t.shape)==2:
            t = t.T[0] #???
        wts[k,:-1] = t
    return wts , wt_pix , nside_wt

In [ ]:
%nbdev_export
def _add_weights(config, wts, wt_pix, nside_wt, photon_data):
    # get the photon pixel ids, convert to NEST (if not already) and right shift them

    if not config.nest:
        # data are RING
        photon_pix = healpy.ring2nest(config.nside, photon_data.pixel.values)
    else:
        photon_pix = photon_data.pixel.values
    to_shift = 2*int(np.log2(config.nside/nside_wt));
    shifted_pix =   np.right_shift(photon_pix, to_shift)
    bad = np.logical_not(np.isin(shifted_pix, wt_pix))
    if config.verbose>0:
        print(f'\tApplyng weights: {sum(bad)} / {len(bad)} photon pixels are outside weight region')
    if sum(bad)==len(bad):
        a = np.array(healpy.pix2ang(nside_wt, wt_pix, nest=True, lonlat=True)).mean(axis=1).round(1)
        b = np.array(healpy.pix2ang(nside_wt, shifted_pix, nest=True, lonlat=True)).mean(axis=1).round(1)

        raise Exception(f'There was no overlap of the photon data at {b} and the weights at {a}')
    shifted_pix[bad] = 12*nside_wt**2 # set index to be beyond pixel indices

    # find indices with search and add a "weights" column
    # (expect that wt_pix are NEST ordering and sorted)
    weight_index = np.searchsorted(wt_pix,shifted_pix)
    band_index = np.fmin(31, photon_data.band.values) #all above 1 TeV into last bin

    # final grand lookup -- isn't numpy wonderful!
    photon_data.loc[:,'weight'] = wts[tuple([band_index, weight_index])]
    if config.verbose>0:
        print(f'\t{sum(np.isnan(photon_data.weight.values))} weights set to NaN')
    

In [ ]:
%nbdev_export
def add_weights(config,  photon_data, source, nbins=50):
    """ add weights for the source to the photon data
    
    - photon_data -- DataFrame with photon data
    
    - source -- `PointSource` object
    
    Return the weight value histogram
    """
    files = config.files

    weight_file =  check_weights(config,  source)
    if weight_file is None:
        raise Exception(f'Weight file not found for {source}')
 
    wts, wt_pix, nside_wt = _load_weights(config, weight_file)
    _add_weights(config, wts, wt_pix, nside_wt, photon_data)

    return np.histogram(photon_data.weight.values, np.linspace(0,1,nbins+1))[0]

In [ ]:
%nbdev_export
def get_weight_hist(config,  source, nbins=50, key=''):
    """ return a weight distribution
        
    - photon_data -- DataFrame with photon data    
    - source -- `PointSource` object
    
    Uses `add_weights`.
    """
    def doit(nbins):
        weight_file =  check_weights(config,  source)
        if weight_file is None:
            raise Exception(f'Weight file not found for {source}')
        photon_data = get_photon_data(config,  source )
        return add_weights(config, photon_data, source, nbins=nbins)
    
    key = f'weight_hist_{source.name}' if key=='' else key
    description = f'Weight histogram for {source.name}' if config.verbose>1 else ''
    return config.cache(key, doit, nbins, description=description)


In [ ]:
# config = Config(wtlike_data='~/wtlike_data')
# if config.valid:
#     source = PointSource('Geminga')
#     h = get_weight_hist(config, source, key='')
#     print(h)

In [ ]:
# %nbdev_collapse_input
# config = Config() 

# if config.valid:
#     source = PointSource('Geminga')
#     print(f'Loading photon data for source {source.name} to test adding weights')
#     photon_data = get_photon_data(config,  source )
#     h = add_weights(config, photon_data, source)
#     print(f'Head of modified photon_data\n {photon_data.head()}')
#     plt.rc('font',size=14)
#     fig, ax =plt.subplots(figsize=(4,3))
#     n = len(h)
#     ax.step( np.linspace(0,1, n+1) , np.concatenate([[h[0]], h])); 
#     ax.grid();
#     ax.set(xlabel='weight', xlim=(0,1), ylim=(0,None))
#     #plt.hist(photon_data.weight.values, np.linspace(0,1,51), histtype='stepfilled', lw=2)
#     plt.title(f'Weights for source {source.name}')
# else:
#     print('Not testing since no files.')

In [ ]:
%nbdev_hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_data_man.ipynb.
Converted 03_source_data.ipynb.
Converted 03_weights.ipynb.
Converted 04_photon_data.ipynb.
Converted 06_poisson.ipynb.
Converted 07_cell_data.ipynb.
Converted 08_loglike.ipynb.
Converted 09_lightcurve.ipynb.
Converted 10_simulation.ipynb.
Converted 14_bayesian.ipynb.
Converted index.ipynb.
Mon May  3 13:42:40 PDT 2021
